<a href="https://colab.research.google.com/github/MinSong1227/aisecurity01/blob/main/ai_security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/데이터.zip

In [129]:
import os
import glob
import json
import pprint
import pandas
import csv

import numpy as np

from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

In [130]:
SEED = 41

def read_label_csv(path):
    label_table = dict()
    with open(path, "r", encoding= "cp949") as f:
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table


def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_jobs=4)
    elif kwargs["model"] == "dt":
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None
    

def train(X_train, y_train, model):
    '''
        머신러닝 모델을 선택하여 학습을 진행하는 함수
	
        :param X_train: 학습할 2차원 리스트 특징벡터
        :param y_train: 학습할 1차원 리스트 레이블 벡터
        :param model: 문자열, 선택할 머신러닝 알고리즘
        :return: 학습된 머신러닝 모델 객체
    '''
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf


def evaluate(X_test, y_test, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_test)
    print("정확도", model.score(X_test, y_test))


## 레이블 테이블 로드

In [131]:
label_table = read_label_csv("학습데이터_정답.csv")
label_table1 = read_label_csv("검증데이터_정답.csv")

## 특징 벡터 생성 예시
- PEMINER 정보는 모두 수치형 데이터이므로 특별히 가공을 하지 않고 사용 가능
- EMBER, PESTUDIO 정보는 가공해서 사용해야 할 특징들이 있음 (e.g. imports, exports 등의 문자열 정보를 가지는 데이터)
- 수치형 데이터가 아닌 데이터(범주형 데이터)를 어떻게 가공할 지가 관건 >> 인코딩 (e.g. 원핫인코딩, 레이블인코딩 등)

In [132]:
class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []


    # def get_general_file_info(self):
    #     vector = [ "CMC.DLL", "Section.text.VirtualSize"]
    #     return vector
    

    def process_report(self):
        
        #     전체 데이터 사용  


        ##1
        # key_list = []

        # for key in self.report.keys():
        #   key_list.append(key)
        ##1

        # vector = random.sample(key_list, 30)
        ##1
        
              
        
        #vector = [ "CMC.DLL", "Section.text.VirtualSize"]
        # temp= []
        # for key in self.report.keys():
        #   if key in vector:
        #     temp.append((key,self.report[key]))
        
        

        self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        # self.vector = [value for _, value in temp]

        #print(self.report.items())

        #self.vector = [i for i in vectors]
        #print(sorted(self.report.items()))
        #print(self.vector)
        #print(len(self.report.keys()))

        return self.vector
        

class EmberParser:
    '''
        예제에서 사용하지 않은 특징도 사용하여 벡터화 할 것을 권장
    '''
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def get_histogram_info(self):
        histogram = np.array(self.report["histogram"])
        total = histogram.sum()
        vector = histogram / total
        return vector.tolist()


    def get_section_info(self):
      entropys = []
      strings = self.report["section"]
      temp = 0
      #temp = self.report["section"]["sections"][
      for i in range(len(strings["sections"])):
        temp = max(temp,strings["sections"][i]["entropy"])
      
      # if len(strings["sections"]):
      #        entropys.append(temp / len(strings["sections"]))
      # else:
      #   entropys.append(0)
      if(temp >= 6.0):
        entropys.append(temp)
      else:
        entropys.append(0)
      # print(entropys)
      return entropys

      

    def get_string_info(self):
        strings = self.report["strings"]
        #strings = self.report["section"]

        hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
        vector = [
            strings['numstrings'], 
            strings['avlength'], 
            strings['printables'],
            strings['entropy'], 
            strings['paths'], 
            strings['urls'],
            strings['registry'], 
            strings['MZ']
        ]
        vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
        return vector
    
    def get_general_file_info(self):
        general = self.report["general"]

        vector = [
            general['size'], general['vsize'], general['has_debug'], general['exports'], general['imports'],
            general['has_relocations'], general['has_resources'], general['has_signature'], general['has_tls'],
            general['symbols']
        ]
        return vector

    def import_info(self):
      imports = self.report["imports"]
      count = 0
      for i in imports:
        count = count + 1
      return [count]


    def get_byte_entropy(self):
      entro = self.report["byteentropy"]
      count = 0        
      for i in entro:
        if i > 100.0:
          count += 1
        
      return [count]

              
    def process_report(self):
        vector = []
        vector += self.get_general_file_info()
        vector += self.get_histogram_info()
        vector += self.get_string_info()
        vector += self.import_info()
        # vector += self.get_section_info()
        vector += self.get_byte_entropy()
        '''
            특징 추가
        '''
        #print(vector)
        return vector
    
# class PestudioParser:
#     '''
#         사용할 특징을 선택하여 벡터화 할 것을 권장
#     '''
    
#     def __init__(self, path):
#         self.report = read_json(path)
#         self.vector = []
    
#     def process_report(self):
#         pass

## 학습데이터 구성
- 특징 벡터 구성은 2차원이 되어야함 e.g.  [vector_1, vector_2, ..., vector_n]

- 각 벡터는 1차원 리스트, 벡터 크기는 모두 같아야함

In [133]:
# 데이터의 특징 벡터 모음(2차원 리스트) : X
from collections import defaultdict
# 데이터의 레이블 모음(1차원 리스트) : y
X, y = [], []
x_validation, y_validation = [],[]
file_list = os.listdir("EMBER/학습데이터")
file_list2 = os.listdir("EMBER/검증데이터")

#file_list = file_list + file_list2+ file_list3
k=[]
m=[]
for a in file_list:
	k.append(a.replace('.json',''))

for b in file_list2:
  m.append(b.replace('.json',''))


for fname in k:
     feature_vector = []
     
     label = label_table[str(fname)]
     for data in ["PEMINER", "EMBER"]:
         path = f"{data}/학습데이터/{fname}.json"
         if data == "PEMINER":
             feature_vector += PeminerParser(path).process_report()
         else:
             feature_vector += EmberParser(path).process_report()

     X.append(feature_vector)
     y.append(label)

for fname in m:
     feature_vector1=[]
     label = label_table1[str(fname)]

     for data in ["PEMINER", "EMBER"]:
       path = f"{data}/검증데이터/{fname}.json"
       if data == "PEMINER":
         feature_vector1 += PeminerParser(path).process_report()
       else:
         feature_vector1 += EmberParser(path).process_report()

     x_validation.append(feature_vector1)
     y_validation.append(label)

print(np.asarray(X).shape, np.asarray(y).shape)
np.asarray(x_validation).shape, np.asarray(y_validation).shape


(20000, 560) (20000,)


((10000, 560), (10000,))

## 학습 및 검증

In [134]:

# 학습
models = []
for model in ["rf","lgb"]:
    clf = train(X, y, model)
    models.append(clf)

# 검증
# 실제 검증 시에는 제공한 검증데이터를 검증에 사용해야 함
for model in models:
    evaluate(x_validation, y_validation, model)

# 정확도 0.9424
# 정확도 0.9517

# 최대 entropy 제거
# 정확도 0.9429
# 정확도 0.9506

정확도 0.9427
정확도 0.9522


## 앙상블 예제

In [135]:
def ensemble_result(X, y, models):
    '''
        학습된 모델들의 결과를 앙상블하는 함수
	
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param models: 1개 이상의 학습된 머신러닝 모델 객체를 가지는 1차원 리스트
    '''
 
    # Soft Voting
    # https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]

        
    print("정확도", accuracy_score(y, predict))

In [136]:
ensemble_result(x_validation, y_validation, models)
#정확도 0.9525
#정확도 0.9527 엔트로피 제거
# 정확도 0.9531 엔트로피 바이트 10.0 이상

정확도 0.9532


# 테스트 데이터 예측 결과 파일 출력

In [137]:
def ensemble_result_predict_csv():
    fin=[]
    file_list2 = os.listdir("EMBER/테스트데이터")
    o=[]
    for a in file_list2:
      o.append(a.replace('.json',''))

    for fname in o:
     feature_vector2=[]

     for data in ["PEMINER", "EMBER"]:
       path = f"{data}/테스트데이터/{fname}.json"
       if data == "PEMINER":
         feature_vector2 += PeminerParser(path).process_report()
       else:
         feature_vector2 += EmberParser(path).process_report()

     fin.append(feature_vector2)

    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(fin)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]


    f = open('predict.csv', 'w', encoding='cp949')
    wr = csv.writer(f)
    wr.writerow(["file","predict"])
    for i in range(len(o)):
      wr.writerow([o[i],predict[i]])
    f.close()

In [138]:
ensemble_result_predict_csv()